# BINOMIAL ASSET PRICING MODEL

Implementation of a simple binomial pricing model in python. We will treat binomial tree as a network with nodes(i,j) with i representing the time steps and j representing the number of ordered price outcome (lowest - or bottom of tree - to highest)

# Generic timing wrapper function
We will use this to benchmark the models

In [3]:
import numpy as np

from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts)) 
        return result
    return wrap

# Binomial Tree Representatin


In [5]:
# Initialise parameters
S0 = 100      # initial stock price
K = 100       # strike price
T = 1         # time to maturity in years
r = 0.06      # annual risk-free rate
N = 3         # number of time steps
u = 1.1       # up-factor in binomial models
d = 1/u       # ensure recombining tree
opttype = 'C' # Option Type 'C' or 'P'

# Binomial Tree Implement (using arrray)
Here we will use for loops to iterate through nodes j at each time step i

In [7]:
@timing
def binomial_tree(K, T, S0, r, N, u, d, opttype = 'C'):
    #precomute constants
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)
    
    # initialize asset prices at maturity - TIme step N
    S = np.zeros(N+1)
    S[0] = S0*d**N
    for j in range(1,N+1):
        S[j] = S[j-1]*u/d
        
    # initialize option values at maturity
    C = np.zeros(N+1)
    for j in range(0,N+1):
        C[j] = max(0, S[j] - K)

    # step backwards through tree
    for i in np.arange(N,0,-1):
        for j in range(0,i):
            C[j] = disc * (q*C[j+1] + (1-q)*C[j])

    return C[0]

binomial_tree(K, T, S0, r, N, u, d, opttype = 'C')

func:'binomial_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0000 sec


10.145735799928817

# Binomial Tree impelement (try to use another faster way)
Now we will vectorise out code using numpy arrays instead of for loops through j nodes.

In [9]:
@timing
def binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C'):
    #precompute constants
    dt = T/N
    q = (np.exp(r*dt) - d) / (u-d)
    disc = np.exp(-r*dt)

    # initialise asset prices at maturity - Time step N
    C = S0 * d ** (np.arange(N,-1,-1)) * u ** (np.arange(0,N+1,1))

    # initialise option values at maturity
    C = np.maximum( C - K , np.zeros(N+1) )

    # step backwards through tree
    for i in np.arange(N,0,-1):
        C = disc * ( q * C[1:i+1] + (1-q) * C[0:i] )

    return C[0]

binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C')

func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0010 sec


10.145735799928826

# Compare 2 methods
Now we will compare runtimes of two models. Ignore option price changes as this is impacted with changing the time steps and keeping the u and d factors the same

In [11]:
for N in [3,50, 100, 1000, 5000]:
    binomial_tree(K,T,S0,r,N,u,d,opttype='C')
    binomial_tree_fast(K,T,S0,r,N,u,d,opttype='C')

func:'binomial_tree' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0000 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 3, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0000 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0010 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 50, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0010 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0020 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 100, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0010 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.2269 sec
func:'binomial_tree_fast' args:[(100, 1, 100, 0.06, 1000, 1.1, 0.9090909090909091), {'opttype': 'C'}] took: 0.0070 sec
func:'binomial_tree' args:[(100, 1, 100, 0.06, 5000, 1.1, 0.9090909090909091), {